# Feature Engineering Part 1

On this Noteboook, the process of Feature Enginnering and Feature Selection is debugged using reduced data. After the definition of the process, all the code generated is applied to the complete data on Part 2.

## Libraries

In [1]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data, transform_data

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

# Feature Selection Libraries
from cnr_methods import LOFO_GPU_Importance
import xgboost as xgb

#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

## Read Data

For this pipeline, only Training Set will be used.

In [2]:
full_data, y_train = get_simplified_data()
train_data = full_data[full_data['Set']=='Train']

In [3]:
train_data.head()

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [4]:
WF = 'WF1'
data = train_data[train_data['WF']==WF]
y_train = y_train[y_train.index.isin(data['ID'])]

In [5]:
data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575000,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train
...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,6235,WF1,-2.038990,-7.244520,-0.077702,-2.412487,280.165000,0.000000,Train
2019-01-15 21:00:00,6236,WF1,-1.453913,-5.145231,-0.291905,-2.148108,280.748363,0.000000,Train
2019-01-15 22:00:00,6237,WF1,-2.378300,-5.825480,-0.559868,-1.926407,279.299000,0.000000,Train


## Feature Creation

First, using the Zonal and Meridional Components of Wind, the Magnitude and Direction of Wind Vector for 100m and 10m height.

### Wind Speed Vector

In [6]:
feature_data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])

Changing Reference for Negative Angles:

In [7]:
feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

Using the original Features, we will create some variables over the Numerical Variables from the simplified data.

In [8]:
features = ['T', 'CLCT', 'U_100m','V_100m','U_10m','V_10m']

### Time-Relative Variables

Here,  Values for Last Week and Month for each Numerical Feature are generated.

In [9]:
for column in features:
    feature_data[column + '_last_week'] = feature_data[column].shift(7)
    feature_data[column + '_last_month'] = feature_data[column].shift(30)

Now, Month and Quarter Statistics(Mean,Median,Variance) are generated:

In [10]:
feature_data['Month_Number'] = feature_data.index.month
feature_data['Quarter_Number'] = feature_data.index.quarter

In [11]:
# Month
mean_month = feature_data.groupby('Month_Number').mean()[features]
median_month = feature_data.groupby('Month_Number').median()[features]
variance_month = feature_data.groupby('Month_Number').var()[features]

# Quarter
mean_quarter = feature_data.groupby('Quarter_Number').mean()[features]
median_quarter = feature_data.groupby('Quarter_Number').median()[features]
variance_quarter = feature_data.groupby('Quarter_Number').var()[features]

In [12]:
# Month
mean_month.columns = mean_month.columns + '_Month_Mean'
median_month.columns = median_month.columns + '_Month_Median'
variance_month.columns = variance_month.columns + '_Month_Variance'

# Quarter
mean_quarter.columns = mean_quarter.columns + '_Quarter_Mean'
median_quarter.columns = median_quarter.columns + '_Quarterh_Median'
variance_quarter.columns = variance_quarter.columns + '_Quarter_Variance'

In [13]:
# Month
feature_data = feature_data.merge(mean_month,on='Month_Number',how='left')
feature_data = feature_data.merge(median_month,on='Month_Number',how='left')
feature_data = feature_data.merge(variance_month,on='Month_Number',how='left')

# Quarter
feature_data = feature_data.merge(mean_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(median_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(variance_quarter,on='Quarter_Number',how='left')

In [14]:
feature_data.index = data.index

In [31]:
feature_data = feature_data.drop(['Month_Number','Quarter_Number'],axis=1)

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set,Wind Speed 100m,...,V_10m_Rolling_7_Window_Mean,V_10m_Rolling_14_Window_Mean,V_10m_Rolling_7_Window_Variance,V_10m_Rolling_14_Window_Variance,T_Expanded_Window_Min,CLCT_Expanded_Window_Min,U_100m_Expanded_Window_Min,V_100m_Expanded_Window_Min,U_10m_Expanded_Window_Min,V_10m_Expanded_Window_Min
Time,,,,,,,,,,,,,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train,3.958410,...,NaN,NaN,NaN,NaN,286.440,82.543144,-2.248500,-3.257800,1.254603,-0.289687
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train,2.831607,...,NaN,NaN,NaN,NaN,286.440,99.990844,-2.248500,-1.446100,2.490908,-0.289687
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575000,98.367235,Train,1.249405,...,NaN,NaN,NaN,NaN,286.575,99.990844,-1.220571,-0.266871,2.490908,-0.289687
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train,7.238384,...,NaN,NaN,NaN,NaN,286.575,99.990844,3.706500,-0.266871,2.490908,-0.289687
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train,6.647232,...,NaN,NaN,NaN,NaN,286.575,99.990844,3.813400,-0.266871,2.490908,-0.289687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,6235,WF1,-2.038990,-7.244520,-0.077702,-2.412487,280.165000,0.000000,Train,7.525992,...,-4.309160,-6.669764,3.569699,7.765535,307.755,100.000000,15.974111,14.380300,10.138381,8.634779
2019-01-15 21:00:00,6236,WF1,-1.453913,-5.145231,-0.291905,-2.148108,280.748363,0.000000,Train,5.346706,...,-3.587709,-6.138899,2.349660,8.383165,307.755,100.000000,15.974111,14.380300,10.138381,8.634779
2019-01-15 22:00:00,6237,WF1,-2.378300,-5.825480,-0.559868,-1.926407,279.299000,0.000000,Train,6.292259,...,-2.957810,-5.627402,1.088174,8.797686,307.755,100.000000,15.974111,14.380300,10.138381,8.634779


For periodical Features, here represented by days (Of Month, Week and Year), hour and minutes, the features are applied to sinusoidal functions to replicate the cyclic nature of the variables.

In [15]:
day = feature_data.index.day
hour = feature_data.index.hour
minute = feature_data.index.minute
dayofweek = feature_data.index.dayofweek
dayofyear = feature_data.index.dayofyear

In [16]:
days_in_month = feature_data.index.days_in_month

In [17]:
feature_data["cos_day"], feature_data["sin_day"] = (
    np.cos(2 * np.pi * (day - 1) / days_in_month),
    np.sin(2 * np.pi * (day - 1) / days_in_month),
    )

feature_data["cos_hour"], feature_data["sin_hour"] = (
    np.cos(2 * np.pi * hour / 24),
    np.sin(2 * np.pi * hour / 24),
    )

feature_data["cos_minute"], feature_data["sin_minute"] = (
    np.cos(2 * np.pi * minute / 60),
    np.sin(2 * np.pi * minute / 60),
)

feature_data["cos_dayofyear"], feature_data["sin_dayofyear"] = (
    np.cos(2 * np.pi * (dayofyear - 1) / 365),
    np.sin(2 * np.pi * (dayofyear - 1) / 365),
)

feature_data["cos_dayofweek"], feature_data["sin_dayofweek"] = (
    np.cos(2 * np.pi * dayofweek / 7),
    np.sin(2 * np.pi * dayofweek / 7),
)

### Distance from Features

Distance of Position of Max and Min (Already on Tsfresh, check it later):

In [18]:
for column in features:
    feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
    feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
    feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
    feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

### Rolling Window Statistics

In [19]:
for column in features:
    feature_data[column + '_Rolling_7_Window_Mean'] = feature_data[column].rolling(window=7).mean()
    feature_data[column + '_Rolling_14_Window_Mean'] = feature_data[column].rolling(window=14).mean()
    feature_data[column + '_Rolling_7_Window_Variance'] = feature_data[column].rolling(window=7).var()
    feature_data[column + '_Rolling_14_Window_Variance'] = feature_data[column].rolling(window=14).var()

### Expanding Window Statistics

In [20]:
for column in features:
    feature_data[column + '_Expanded_Window_Min'] = feature_data[column].expanding().min()
    feature_data[column + '_Expanded_Window_Min'] = feature_data[column].expanding().max()

## Feature Selection

Finally, all the Features Generated have to be filtered, so only the most relevant ones are passed to the model, as a way of avoiding Overfitting. For this process, a process called LOFO (Leave One Feature Out) is selected.

The process is simple: For each Feature, a arbitrary Model (Here a XGBoost) is cross validated on a dataset that contains all the features except one, which importance is being measured, and the precision without that feature is compared to a baseline, where all features are present. The features whose removal leads to worse results are considered most important features to the dataset.

More info about this process can be found on https://github.com/aerdem4/lofo-importance.

The implementation of this method on this however, was an adaptation of the original code to allow use of GPU resources as a faster way to obtain results, since this work involves a big number of columns.

In [21]:
final_features = feature_data.drop(features,axis=1)
final_features = final_features.rename({'key_0':'Date'},axis=1)

In [22]:
features = final_features.drop(['ID','WF','Set'],axis=1).columns

In [23]:
param = {'tree_method' : 'gpu_hist'}

In [24]:
importance_df = LOFO_GPU_Importance(final_features,y_train,features,param)

1/106 2.693571 s/it
2/106 3.464999 s/it
3/106 2.786999 s/it
4/106 3.84625 s/it
5/106 2.559002 s/it
6/106 2.807000 s/it
7/106 2.962514 s/it
8/106 3.102801 s/it
9/106 2.804033 s/it
10/106 2.926967 s/it
11/106 3.794001 s/it
12/106 3.128998 s/it
13/106 3.572027 s/it
14/106 3.155082 s/it
15/106 3.741181 s/it
16/106 3.699075 s/it
17/106 3.31000 s/it
18/106 3.95178 s/it
19/106 3.109073 s/it
20/106 3.72220 s/it
21/106 3.161000 s/it
22/106 3.131001 s/it
23/106 3.134537 s/it
24/106 3.92569 s/it
25/106 3.92267 s/it
26/106 3.128243 s/it
27/106 3.71130 s/it
28/106 3.71569 s/it
29/106 3.86203 s/it
30/106 3.98574 s/it
31/106 3.105749 s/it
32/106 3.116589 s/it
33/106 3.54819 s/it
34/106 3.71000 s/it
35/106 3.52000 s/it
36/106 3.120510 s/it
37/106 3.72979 s/it
38/106 3.88108 s/it
39/106 3.77000 s/it
40/106 3.76000 s/it
41/106 3.48000 s/it
42/106 3.102107 s/it
43/106 3.103943 s/it
44/106 3.48003 s/it
45/106 3.67509 s/it
46/106 3.70267 s/it
47/106 3.69508 s/it
48/106 3.72002 s/it
49/106 3.52089 s/it
50/1

In [25]:
importance_df.to_csv(r'C:\Users\andre_\OneDrive\Documentos\Feature Selection\Importance_WF1.csv')

In [26]:
importance_df

,feature,score
3,Wind Direction 10m,-1.880883
15,V_10m_last_month,-0.748472
98,V_10m_Rolling_7_Window_Variance,-0.523809
7,CLCT_last_month,-0.454566
91,V_100m_Rolling_14_Window_Variance,-0.406567
...,...,...
0,Wind Speed 100m,0.657650
92,U_10m_Rolling_7_Window_Mean,0.800479
83,CLCT_Rolling_14_Window_Variance,0.826343
89,V_100m_Rolling_14_Window_Mean,1.154205
